In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from pathlib import Path
import tensorflow_datasets as tfds

SyntaxError: invalid syntax (176715795.py, line 6)

In [ ]:
!pip install --upgrade tfds-nightly
!pip install tensorflow_datasets

In [ ]:
transformer_results = pd.read_csv("/scratch/ajk4yq/ecg/models/transformer-age-history.csv")

In [ ]:
cnn_results = pd.read_csv("/scratch/ajk4yq/ecg/models/cnn-age-history.csv")

In [ ]:
resnet_results = pd.read_csv("/scratch/ajk4yq/ecg/models/resnet-age-history.csv")

In [ ]:
resnet2_results = pd.read_csv("/scratch/ajk4yq/ecg/models/resnet-age2-history.csv")

In [ ]:
transformer_results['Model'] = ['Transformer']*10

In [ ]:
results = [transformer_results, cnn_results, resnet_results, resnet2_results]
labels = ['Transformer', 'CNN', 'ResNet', 'ResNet 2']

In [ ]:
for table, label in zip(results,labels):
    if label == 'Transformer':
        repeat = 10
    else: repeat = 50
    newcol = [label]*repeat
    table['Model'] = newcol

In [ ]:
transformer_results

In [ ]:
all_results = pd.concat([transformer_results, cnn_results, resnet_results, resnet2_results])

In [ ]:
all_results

In [ ]:
pivot_val = all_results.pivot(columns = 'Model', values='val_mse')
pivot_train = all_results.pivot(columns = 'Model', values='mse')
plt.plot(pivot_val)
plt.plot(pivot_train, linestyle='dashed')
plt.legend()

In [ ]:
pivot_val.plot(xlabel='Epochs',ylabel='Validation MSE', title='Validation MSE')
pivot_train.plot(xlabel='Epochs',ylabel='Training MSE', title='Training MSE', linestyle='dashed')

In [ ]:
DATA_PATH = Path("/scratch/ajk4yq/ecg/tfrecords/")
ALL_RECS = list(DATA_PATH.glob("*.tfrecords"))

In [ ]:
BATCH_SIZE = 512

record_format = {
    'ecg/data': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'file_name': tf.io.FixedLenFeature([], tf.int64),
    'hospital_expire_flag': tf.io.FixedLenFeature([], tf.float32),
}

def _parse_record(record):
    example = tf.io.parse_single_example(record, record_format)
    ecg_data = tf.reshape(example['ecg/data'], [5000,12])
    return ecg_data, example['file_name']

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [ ]:
all_recs = get_dataset(ALL_RECS)

In [ ]:
all_recs_df = tf.Data